In [1]:
import marqo

In [2]:
mq = marqo.Client(url="http://localhost:8882")

In [10]:
mq.delete_index("test_index")
settings = {
    "treatUrlsAndPointersAsImages": False,
    "model": "no_model",
    "modelProperties": {
        "dimensions": 1024,
        "type": "no_model",
    },
}
mq.create_index(
    index_name="test_index",
    settings_dict=settings,
)

{'acknowledged': True, 'index': 'test_index'}

In [9]:
mq.index("test_index").add_documents(
    [
        {
            "id": "0",
            "phrase": "partielle hypertonie",
            "phrase_vector": {
                "vector": []
            }
        },
        {
            "id": "1",
            "phrase": "akute psychose",
            "phrase_vector": {
                "vector": []
            }
        },
        {
            "id": "2",
            "phrase": "lungenentzündung",
            "phrase_vector": {
                "vector": []
            }
        }
    ],
    tensor_fields=["phrase_vector"],
    mappings={
        "phrase_vector": {
            "type": "custom_vector"
        }
    },
)

{'errors': False,
 'processingTimeMs': 20113.35337534547,
 'index_name': 'test_index',
 'items': [{'status': 200, '_id': 'c4284a1e-7d40-467e-aced-d752569a6abe'},
  {'status': 200, '_id': '410d423c-3e49-432f-9c3c-b2d9d3305ea4'},
  {'status': 200, '_id': '9c9637b4-1667-4c0b-85e1-5e5bc272c135'}]}